set up the environment using the OpenAI Gym library. I will use the 'CartPole-v1' environment, a common benchmark for reinforcement learning algorithms.


In [16]:
%pip install gym

Note: you may need to restart the kernel to use updated packages.


In [17]:
# !pip install --upgrade numpy==1.26.4
# !pip uninstall tensorflow -y 
# !pip install tensorflow==2.16.2 

Increase the recursion limit, although this is generally more of a workaround than a solution.


In [18]:
import gym
import sys 
sys.setrecursionlimit(1500) 

import gym 
import numpy as np 

# Create the environment 
env = gym.make('CartPole-v1') 

# Set random seed for reproducibility 
np.random.seed(42) 
env.action_space.seed(42) 
env.observation_space.seed(42)

[42]

Define a neural network using Keras to approximate the Q-values. The network will take the state as input and output Q-values for each action.


In [19]:
# Suppress warnings for a cleaner notebook or console experience
import warnings
warnings.filterwarnings('ignore')

# Override the default warning function
def warn(*args, **kwargs):
    pass
warnings.warn = warn

# Import necessary libraries for the Q-Learning model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input  # Import Input layer
from tensorflow.keras.optimizers import Adam
import gym  # Ensure the environment library is available

# Define the model building function
def build_model(state_size, action_size): 
    model = Sequential() 
    model.add(Input(shape=(state_size,)))  # Use Input layer to specify the input shape 
    model.add(Dense(24, activation='relu')) 
    model.add(Dense(24, activation='relu')) 
    model.add(Dense(action_size, activation='linear')) 
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001)) 
    return model 

# Create the environment and set up the model
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0] 
action_size = env.action_space.n 
model = build_model(state_size, action_size)


Implement the Q-Learning algorithm, which involves interacting with the environment, updating the Q-values, and training the neural network. 

**Define the replay Function**


In [20]:
import random
import numpy as np
from collections import deque
import tensorflow as tf

# Define epsilon and epsilon_decay
epsilon = 1.0  # Starting with a high exploration rate
epsilon_min = 0.01  # Minimum exploration rate
epsilon_decay = 0.99  # Faster decay rate for epsilon after each episode

# Replay memory
memory = deque(maxlen=2000)

def remember(state, action, reward, next_state, done):
    """Store experience in memory."""
    memory.append((state, action, reward, next_state, done))

def replay(batch_size=64):  # Increased batch size
    """Train the model using a random sample of experiences from memory."""
    if len(memory) < batch_size:
        return  # Skip replay if there's not enough experience

    minibatch = random.sample(memory, batch_size)  # Sample a random batch from memory
    
    # Extract information for batch processing
    states = np.vstack([x[0] for x in minibatch])
    actions = np.array([x[1] for x in minibatch])
    rewards = np.array([x[2] for x in minibatch])
    next_states = np.vstack([x[3] for x in minibatch])
    dones = np.array([x[4] for x in minibatch])
    
    # Predict Q-values for the next states in batch
    q_next = model.predict(next_states)
    # Predict Q-values for the current states in batch
    q_target = model.predict(states)
    
    # Vectorized update of target values
    for i in range(batch_size):
        target = rewards[i]
        if not dones[i]:
            target += 0.95 * np.amax(q_next[i])  # Update Q value with the discounted future reward
        q_target[i][actions[i]] = target  # Update only the taken action's Q value
    
    # Train the model with the updated targets in batch
    model.fit(states, q_target, epochs=1, verbose=0)  # Train in batch mode

    # Reduce exploration rate (epsilon) after each training step
    global epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

def act(state):
    """Choose an action based on the current state and exploration rate."""
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)  # Explore: choose a random action
    act_values = model.predict(state)  # Exploit: predict action based on the state
    return np.argmax(act_values[0])  # Return the action with the highest Q-value

# Define the number of episodes you want to train the model for
episodes = 10  # You can set this to any number you prefer
train_frequency = 5  # Train the model every 5 steps

for e in range(episodes):
    state, _ = env.reset()  # Unpack the tuple returned by env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(200):  # Limit to 200 time steps per episode
        action = act(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        remember(state, action, reward, next_state, done)  # Store experience
        state = next_state
        
        if done:
            print(f"episode: {e+1}/{episodes}, score: {time}, e: {epsilon:.2}")
            break
        
        # Train the model every 'train_frequency' steps
        if time % train_frequency == 0:
            replay(batch_size=64)  # Call replay with larger batch size for efficiency

env.close()


episode: 1/10, score: 16, e: 1.0
episode: 2/10, score: 15, e: 1.0
2/2 [==============================] - 0s 3ms/step
episode: 3/10, score: 32, e: 0.99
2/2 [==============================] - 0s 4ms/step
episode: 4/10, score: 26, e: 0.93
1/1 [==============================] - 0s 20ms/step
episode: 5/10, score: 15, e: 0.9
1/1 [==============================] - 0s 21ms/step
episode: 6/10, score: 15, e: 0.88
2/2 [==============================] - 0s 4ms/step
episode: 7/10, score: 28, e: 0.83
2/2 [==============================] - 0s 3ms/step
episode: 8/10, score: 17, e: 0.79
1/1 [==============================] - 0s 19ms/step
episode: 9/10, score: 15, e: 0.77
2/2 [==============================] - 0s 3ms/step
episode: 10/10, score: 36, e: 0.71


Ealuate the performance of the trained Q-Learning agent.


In [21]:
for e in range(10):  

    state, _ = env.reset()  # Unpack the state from the tuple 
    state = np.reshape(state, [1, state_size])  # Reshape the state correctly 
    for time in range(500):  
        env.render()  
        action = np.argmax(model.predict(state)[0])  
        next_state, reward, terminated, truncated, _ = env.step(action)  # Unpack the five return values 
        done = terminated or truncated  # Check if the episode is done 
        next_state = np.reshape(next_state, [1, state_size])  
        state = next_state  
        if done:  
            print(f"episode: {e+1}/10, score: {time}")  
            break  

env.close() 

1/1 [==============================] - 0s 18ms/step
episode: 1/10, score: 97
1/1 [==============================] - 0s 33ms/step
episode: 2/10, score: 63
1/1 [==============================] - 0s 18ms/step
episode: 3/10, score: 72
1/1 [==============================] - 0s 17ms/step
episode: 4/10, score: 58
1/1 [==============================] - 0s 24ms/step
episode: 5/10, score: 58
1/1 [==============================] - 0s 20ms/step
episode: 6/10, score: 60
1/1 [==============================] - 0s 37ms/step
episode: 7/10, score: 82
1/1 [==============================] - 0s 42ms/step
episode: 8/10, score: 55
1/1 [==============================] - 0s 24ms/step
episode: 9/10, score: 87
1/1 [==============================] - 0s 30ms/step
episode: 10/10, score: 80


Experiment with Different Network Architectures 

1. Modify the `build_model()` function to include a different number of neurons and layers. For example, increase the number of layers to 3 and the number of neurons in each layer to 64. 
2. Train the model with the modified architecture and observe the performance in terms of average score achieved over 100 episodes. 
3. Compare the performance with the original architecture.


In [22]:
# Define the model_build function
def build_model(state_size, action_size):
    model = Sequential()
    # The input_shape is defined in the first Dense layer for a Sequential model
    model.add(Dense(64, input_shape=(state_size,), activation='relu'))
    model.add(Dense(64, activation='relu')) # Added another layer and increased neurons
    model.add(Dense(64, activation='relu')) # Added another layer with increased neurons
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model
    

In [23]:
# Create the env and set up the model with modified architecture
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
model = build_model(state_size, action_size)

# Reset epsilon and memory for a new training run 
epsilon = 1.0
memory = deque(maxlen=2000)

# Define the number of episodes for training 
episodes = 20  # Increased number of training episodes
train_frequency = 5

In [24]:
# Training the model with the modified architecture
for e in range(episodes):
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size]) # Reshape initial state for model input
    
    for time in range(500): # Increased ceiling for steps per episode during training 
        # The 'act' function, 'remember' function, and 'replay' function
        # must be defined elsewhere in your code for this snippet to run.
        action = act(state) 
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        reward = reward if not done else -10
        remember(state, action, reward, next_state, done)
        
        # Reshape next_state before assigning it to state for the next iteration
        state = np.reshape(next_state, [1, state_size]) 

        if done:
            if (e + 1) % 10 == 0: # Print progress every 10 episodes
                print(f"Training episode: {e+1}/{episodes}, score: {time}, e: {epsilon:.2f}")
            break # End episode if done

        if time % train_frequency == 0:
            replay(batch_size=64)

env.close()


2/2 [==============================] - 0s 5ms/step
Training episode: 10/20, score: 11, e: 0.70
1/1 [==============================] - 0s 31ms/step
Training episode: 20/20, score: 10, e: 0.53


Implement an Adaptive Exploration Rate 


1. Modify the `epsilon` decay strategy to decrease more rapidly when the agent's performance improves significantly. 
2. Implement a check to reduce `epsilon` faster if the agent achieves a score greater than a certain threshold (e.g., 200) in consecutive episodes. 
3. Observe the effect on the learning rate and the agent's performance. 


In [30]:
# Function to adjust epsilon based on performance
def adjust_epsilon(score, consecutive_success_threshold=200):
    global epsilon 

    if score >= consecutive_success_threshold: 
        epsilon = max(epsilon_min, epsilon * 0.9)  # Reduce epsilon faster if performance is good
    else: 
        epsilon = max(epsilon_min, epsilon * epsilon_decay)  # Regular epsilon decay

episodes = 2  # Set number of episodes for training

# Define hyperparameters
batch_size = 32  # A common value for batch size in RL


# Train the model with adaptive epsilon decay
for e in range(episodes): 
    state = env.reset()  
    state = state[0]  # Extract the first element, which is the actual state array
    state = np.reshape(state, [1, len(state)])  # Reshape state to match the expected input shape

    total_reward = 0 

    for time in range(500):  # Limit the episode to 500 time steps
        action = act(state)  # Choose action based on policy
        next_state, reward, done, truncated, _ = env.step(action)  # Unpack 5 values

        reward = reward if not done else -10  # Penalize for reaching a terminal state
        total_reward += reward  # Accumulate rewards

        next_state = np.reshape(next_state, [1, len(next_state)])  # Reshape next state 

        remember(state, action, reward, next_state, done)  # Store experience in memory
        state = next_state  # Update the current state

        if done or truncated:  # Check if the episode is done or truncated
            adjust_epsilon(total_reward)  # Adjust epsilon based on the total reward
            print(f"episode: {e}/{episodes}, score: {time}, e: {epsilon:.2}")  # Print the episode details
            break  # Break out of the loop if the episode is done or truncated

        if len(memory) > batch_size:  # Check if enough experiences are stored in memory
            replay(batch_size)  # Train the model with the stored experiences (pass batch_size here)

1/1 [==============================] - 0s 18ms/step
episode: 0/2, score: 10, e: 0.29
1/1 [==============================] - 0s 21ms/step
episode: 1/2, score: 9, e: 0.27


Implement a Custom Reward Function 


1. Modify the reward function to provide more granular feedback to the agent. For example, give higher rewards for keeping the pole more vertical and closer to the center. 
2. Implement a reward function that rewards the agent proportionally to the angle of the pole and the distance of the cart from the center. 
3. Train the agent with the new reward function and compare the learning speed and stability to the original setting.


In [31]:
# Define a custom reward function based on the cart position and pole angle
def custom_reward(state):
    # Extract state variables: x (cart position), x_dot (cart velocity), theta (pole angle), theta_dot (pole angular velocity)
    x, x_dot, theta, theta_dot = state
    # Custom reward function: Encourage the agent to keep the cart near the center and the pole upright
    reward = (1 - abs(x) / 2.4) + (1 - abs(theta) / 0.20948)
    return reward

episodes = 2  # Number of episodes to run

# Train the model with the custom reward function
for e in range(episodes): 
    state = env.reset()  # Reset the environment

    # Print the state structure for debugging
    print(f"State: {state}, State Type: {type(state)}")
    # Extract the state if it's a tuple and reshape if necessary
    if isinstance(state, tuple):
        state = state[0]  # Extract the first element if it's a tuple

    state = np.reshape(state, [1, state_size])  # Reshape state to match the expected input shape
    for time in range(500):  # Limit the episode to 500 time steps
        action = act(state)  # Choose an action based on the current state
        # Unpack 5 values returned by env.step(action)
        next_state, reward, done, truncated, _ = env.step(action)
        # Compute the custom reward based on the next state
        reward = custom_reward(next_state) if not done else -10

        # Reshape next_state if necessary
        if isinstance(next_state, tuple):
            next_state = next_state[0]  # Extract the first element if it's a tuple

        next_state = np.reshape(next_state, [1, state_size])  # Reshape next state to match input shape
        # Store the experience in memory
        remember(state, action, reward, next_state, done)
        state = next_state  # Update the current state

        if done or truncated:  # If the episode is done, break out of the loop
            print(f"episode: {e}/{episodes}, score: {time}, e: {epsilon:.2}")
            break

        if len(memory) > batch_size:  # If there are enough samples in memory, train the model
            replay(batch_size)  # Train the model with a batch of experiences

State: (array([ 0.04507792, -0.01187342,  0.00889581,  0.0477067 ], dtype=float32), {}), State Type: <class 'tuple'>
1/1 [==============================] - 0s 19ms/step
episode: 0/2, score: 8, e: 0.24
State: (array([ 0.0377818 ,  0.03064548, -0.01370763, -0.03871557], dtype=float32), {}), State Type: <class 'tuple'>
1/1 [==============================] - 0s 18ms/step
episode: 1/2, score: 9, e: 0.22
